In [ ]:
!pip install statsbombpy
!pip install mplsoccer
!pip install highlight_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 2.6 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="statsbombpy.api_client")

In [ ]:
import pandas as pd
from statsbombpy import sb

# Encontrar los partidos que inclyen goles tempranos





In [ ]:
comps = []
competitions = sb.competitions()
for index, row in competitions.iterrows():
    competition_name = row['competition_name']
    season_name = row['season_name']
    comp_id = row['competition_id']
    season_id = row['season_id']
    matches = sb.matches(competition_id=row['competition_id'], season_id=row['season_id'])
    match_total = len(matches)
    if ( (("Euro" in competition_name or "World Cup" in competition_name or
        "Copa America" in competition_name or "Cup of Nations" in competition_name) and match_total > 10) or
        match_total > 50):
        print(f"Competition: {competition_name}, Season: {season_name}, Match Total: {match_total}")
        info = {
            'competition_name': competition_name,
            'season_name': season_name,
            'comp_id': comp_id,
            'season_id': season_id}

        comps.append(info)

Competition: 1. Bundesliga, Season: 2015/2016, Match Total: 306
Competition: African Cup of Nations, Season: 2023, Match Total: 52
Competition: Copa America, Season: 2024, Match Total: 32
Competition: FA Women's Super League, Season: 2020/2021, Match Total: 131
Competition: FA Women's Super League, Season: 2019/2020, Match Total: 87
Competition: FA Women's Super League, Season: 2018/2019, Match Total: 108
Competition: FIFA World Cup, Season: 2022, Match Total: 64
Competition: FIFA World Cup, Season: 2018, Match Total: 64
Competition: Indian Super league, Season: 2021/2022, Match Total: 115
Competition: La Liga, Season: 2015/2016, Match Total: 380
Competition: Ligue 1, Season: 2015/2016, Match Total: 377
Competition: Premier League, Season: 2015/2016, Match Total: 380
Competition: Serie A, Season: 2015/2016, Match Total: 380
Competition: UEFA Euro, Season: 2024, Match Total: 51
Competition: UEFA Euro, Season: 2020, Match Total: 51
Competition: UEFA Women's Euro, Season: 2022, Match Tota

In [ ]:
EG_match_data = []
match_data = []

counter = 0

print(len(comps))
for competition in comps:
    comp_id = competition['comp_id']
    season_id = competition['season_id']
    competition_name = competition['competition_name']
    season_name = competition['season_name']

    print(f"Processing Competition: {competition_name}, Season: {season_name}")

    matches = sb.matches(competition_id=comp_id, season_id=season_id)
    print(len(matches))

    for _, match_d in matches.iterrows():
        match_id = match_d['match_id']
        home_team = match_d['home_team']
        away_team = match_d['away_team']
        match_date = match_d['match_date']

        try:
            events = sb.events(match_id=match_id, flatten_attrs=True)
        except Exception as e:
            print(f"Error retrieving events for match_id {match_id}: {e}")
            continue

        events['minute'] = pd.to_numeric(events['minute'], errors='coerce').fillna(0).astype(int)
        events['second'] = pd.to_numeric(events['second'], errors='coerce').fillna(0).astype(int)


        kick_off_events = events[events['type'] == 'Half Start']

        kick_off_times = kick_off_events['minute'] * 60 + kick_off_events['second']

        goals = events[(events['type'] == 'Shot') & (events['shot_outcome'] == 'Goal')]

        is_within_3_minutes = False

        for _, goal in goals.iterrows():

            goal_time = goal['minute'] * 60 + goal['second']

            half = goal['period']

            for kick_off_time in kick_off_times.unique():
                time_difference = goal_time - kick_off_time
                if 0 <= time_difference <= 240:

                    if (kick_off_time == 0 and half == 1) or (kick_off_time > 0 and half == 2):
                        is_within_3_minutes = True
                        break

        print(match_id, home_team, away_team, is_within_3_minutes, len(events))
        match_info = {
                'competition_id': comp_id,
                'competition_name': competition_name,
                'season_id': season_id,
                'season_name': season_name,
                'match_id': match_id,
                'match_date': match_date,
                'home_team': home_team,
                'away_team': away_team
            }

        match_data.append(match_info)

        if is_within_3_minutes:
          print(match_info)

          EG_match_data.append(match_info)

df_matches_with_early_goals = pd.DataFrame(EG_match_data)

18
Processing Competition: 1. Bundesliga, Season: 2015/2016
306
3890561 Hoffenheim Schalke 04 False 3940
3890505 Bayern Munich Eintracht Frankfurt False 3571
3890511 Hertha Berlin Hannover 96 True 3395
{'competition_id': 9, 'competition_name': '1. Bundesliga', 'season_id': 27, 'season_name': '2015/2016', 'match_id': 3890511, 'match_date': '2016-04-08', 'home_team': 'Hertha Berlin', 'away_team': 'Hannover 96'}
3890515 Hamburger SV Darmstadt 98 False 3225
3890411 Hertha Berlin FSV Mainz 05 False 3672
3890397 Hoffenheim Hannover 96 False 3336
3890401 Borussia Dortmund Eintracht Frankfurt False 3889
3890396 Wolfsburg Hamburger SV False 3774
3890384 Borussia Dortmund VfB Stuttgart True 3735
{'competition_id': 9, 'competition_name': '1. Bundesliga', 'season_id': 27, 'season_name': '2015/2016', 'match_id': 3890384, 'match_date': '2015-11-29', 'home_team': 'Borussia Dortmund', 'away_team': 'VfB Stuttgart'}
3890385 Schalke 04 Hannover 96 False 3272
3890282 FSV Mainz 05 Hannover 96 True 3361
{'c

In [ ]:
EG_matches = df_matches_with_early_goals
EG_matches['match_date'] = pd.to_datetime(EG_matches['match_date'], errors='coerce')
EG_matches

,competition_id,competition_name,season_id,season_name,match_id,match_date,home_team,away_team
0,9,1. Bundesliga,27,2015/2016,3890511,2016-04-08,Hertha Berlin,Hannover 96
1,9,1. Bundesliga,27,2015/2016,3890384,2015-11-29,Borussia Dortmund,VfB Stuttgart
2,9,1. Bundesliga,27,2015/2016,3890282,2015-08-29,FSV Mainz 05,Hannover 96
3,9,1. Bundesliga,27,2015/2016,3890540,2016-04-30,Bayer Leverkusen,Hertha Berlin
4,9,1. Bundesliga,27,2015/2016,3890535,2016-04-23,Wolfsburg,Augsburg
...,...,...,...,...,...,...,...,...
423,72,Women's World Cup,30,2019,69208,2019-06-29,Germany Women's,Sweden Women's
424,72,Women's World Cup,30,2019,22948,2019-06-12,France Women's,Norway Women's
425,72,Women's World Cup,30,2019,68338,2019-06-19,Scotland Women's,Argentina Women's
426,72,Women's World Cup,30,2019,69191,2019-06-25,Italy Women's,China PR Women's


In [ ]:
EG_matches.to_csv('/content/drive/MyDrive/Hackathon FUT/Data/EG_matches.csv', index=False)

In [ ]:
EG_all_matches = pd.DataFrame(match_data)
EG_all_matches['match_date'] = pd.to_datetime(EG_all_matches['match_date'], errors='coerce')
EG_all_matches.to_csv('/content/drive/MyDrive/Hackathon FUT/Data/EG_all_matches.csv', index=False)

# Extraer los eventos antes del gol de cada partido junto con datos de rendimiento de otros partidos

In [ ]:
EG_matches = pd.read_csv('/content/drive/MyDrive/Hackathon FUT/Data/EG_matches.csv')

In [ ]:
match_id = EG_matches['match_id'].iloc[0]
events = sb.events(match_id=match_id)
for col in events.columns:
  print(col)

50_50
ball_receipt_outcome
ball_recovery_recovery_failure
block_offensive
carry_end_location
clearance_aerial_won
clearance_body_part
clearance_head
clearance_left_foot
clearance_right_foot
counterpress
dribble_outcome
dribble_overrun
duel_outcome
duel_type
duration
foul_committed_advantage
foul_committed_card
foul_committed_type
foul_won_advantage
foul_won_defensive
goalkeeper_body_part
goalkeeper_end_location
goalkeeper_outcome
goalkeeper_position
goalkeeper_technique
goalkeeper_type
id
index
interception_outcome
location
match_id
minute
miscontrol_aerial_won
off_camera
out
pass_aerial_won
pass_angle
pass_assisted_shot_id
pass_body_part
pass_cross
pass_cut_back
pass_end_location
pass_goal_assist
pass_height
pass_inswinging
pass_length
pass_outcome
pass_outswinging
pass_recipient
pass_recipient_id
pass_shot_assist
pass_straight
pass_switch
pass_technique
pass_through_ball
pass_type
period
play_pattern
player
player_id
position
possession
possession_team
possession_team_id
related_even

In [ ]:
import warnings
import pandas as pd
from statsbombpy import sb
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime

In [ ]:
team_matches = defaultdict(list)

for _, match in EG_all_matches.iterrows():
    match_date = match['match_date']
    home_team = match['home_team']
    away_team = match['away_team']
    match_id = match['match_id']

    team_matches[home_team].append((match_date, match_id))
    team_matches[away_team].append((match_date, match_id))

for team in team_matches:
    team_matches[team] = sorted(team_matches[team], key=lambda x: x[0])

In [ ]:
def calculate_team_metrics(events, home_team, away_team):

    metrics = {
        home_team: {
            'GF': 0,
            'GA': 0,
            'Shots_on_Target': 0,
            'Shots_off_Target': 0,
            'Possession_Percentage': 0.0,
            'Pass_Accuracy_Percentage': 0.0,
            'xG': 0.0,
            'xGA': 0.0
        },
        away_team: {
            'GF': 0,
            'GA': 0,
            'Shots_on_Target': 0,
            'Shots_off_Target': 0,
            'Possession_Percentage': 0.0,
            'Pass_Accuracy_Percentage': 0.0,
            'xG': 0.0,
            'xGA': 0.0
        }
    }

    teams = [home_team, away_team]
    opponents = {home_team: away_team, away_team: home_team}

    shots = events[events['type'] == 'Shot']

    for _, shot in shots.iterrows():
        team = shot['team']
        if team not in teams:
            continue

        if shot['shot_outcome'] == 'Goal':
            metrics[team]['GF'] += 1
            metrics[team]['xG'] += shot.get('shot_statsbomb_xg', 0) or 0
        else:
            metrics[team]['xG'] += shot.get('shot_statsbomb_xg', 0) or 0

        shot_outcome = shot['shot_outcome']
        if shot_outcome in ['Goal', 'Saved', 'Blocked']:
            metrics[team]['Shots_on_Target'] += 1
        else:
            metrics[team]['Shots_off_Target'] += 1


    for team in teams:
        opponent = opponents[team]
        opponent_goals = shots[(shots['team'] == opponent) & (shots['shot_outcome'] == 'Goal')]
        metrics[team]['GA'] = opponent_goals.shape[0]

        opponent_shots = shots[shots['team'] == opponent]
        metrics[team]['xGA'] = opponent_shots['shot_statsbomb_xg'].sum() if not opponent_shots.empty else 0.0


    if 'possession' in events.columns and 'possession_team' in events.columns:
        possession = events[['possession', 'possession_team']].dropna()
        total_possession_time = possession['possession'].sum()

        for team in teams:
            team_possession_time = possession[possession['possession_team'] == team]['possession'].sum()
            metrics[team]['Possession_Percentage'] = (team_possession_time / total_possession_time) * 100 if total_possession_time > 0 else 0.0
    else:
        for team in teams:
            metrics[team]['Possession_Percentage'] = float('nan')


    passes = events[events['type'] == 'Pass']

    for team in teams:
        team_passes = passes[passes['team'] == team]
        total_passes = team_passes.shape[0]
        successful_passes = team_passes[team_passes['pass_outcome'].isnull()].shape[0]

        metrics[team]['Pass_Accuracy_Percentage'] = (successful_passes / total_passes) * 100 if total_passes > 0 else 0.0

    return metrics


In [ ]:
def get_surrounding_matches(team, reference_date, num_matches=5):

    matches = team_matches.get(team, [])
    previous = []
    next_matches = []

    if isinstance(reference_date, str):
        reference_date = datetime.strptime(reference_date, '%Y-%m-%d')

    for i, (match_date, match_id) in enumerate(matches):
        if match_date >= reference_date:
            break
        else:
            i = len(matches)

    previous = matches[max(i - num_matches, 0):i]

    next_matches = matches[i:i + num_matches]

    return {
        'previous_matches': [match_id for _, match_id in previous],
        'next_matches': [match_id for _, match_id in next_matches]
    }

In [ ]:
goal_data = []
goal_event_data = []
team_metrics_data = []

for _, match_info in EG_matches.iterrows():
    match_id = match_info['match_id']
    print(f"Processing match_id: {match_id}")
    print(f"team: {match_info['home_team']}")
    print(f"team: {match_info['away_team']}")

    try:
        events = sb.events(match_id=match_id, flatten_attrs=True)
    except Exception as e:
        print(f"Error retrieving events for match_id {match_id}: {e}")
        continue

    events['minute'] = pd.to_numeric(events['minute'], errors='coerce').fillna(0).astype(int)
    events['second'] = pd.to_numeric(events['second'], errors='coerce').fillna(0).astype(int)

    kick_off_events = events[events['type'] == 'Half Start']
    kick_off_times = kick_off_events['minute'] * 60 + kick_off_events['second']

    goals = events[(events['type'] == 'Shot') & (events['shot_outcome'] == 'Goal')]

    first_pass_events = events[events['type'] == 'Pass'].groupby('period').first()

    for _, goal in goals.iterrows():
        goal_time = goal['minute'] * 60 + goal['second']
        is_within_5_minutes = False
        half = goal['period']

        team_scored = goal['team']
        xg = goal['shot_statsbomb_xg']

        for kick_off_time in kick_off_times:
            time_difference = goal_time - kick_off_time
            if 0 <= time_difference <= 300:
                if (kick_off_time == 0 and half == 1) or (kick_off_time > 0 and half == 2):
                    is_within_5_minutes = True
                    break

        if half in first_pass_events.index:
            team_started_half = first_pass_events.loc[half, 'team']
            is_kickoff_team = team_scored == team_started_half
        else:
            is_kickoff_team = False

        preceding_events = events[
            (events['period'] == half) &
            ((events['minute'] * 60 + events['second']) <= goal_time) &
            (events['player'].notnull())
        ]

        if is_within_5_minutes:
            goal_info = {
                'match_id': match_id,
                'goal_minute': goal['minute'],
                'competition_name': match_info['competition_name'],
                'season_name': match_info['season_name'],
                'half': half,
                'home_team': match_info['home_team'],
                'away_team': match_info['away_team'],
                'scoring_team': team_scored,
                'xg': xg,
                'is_kickoff_team': is_kickoff_team
            }
            goal_data.append(goal_info)

            for _, event in preceding_events.iterrows():
                event_info = {
                    'match_id': match_id,
                    'competition_name': match_info['competition_name'],
                    'season_name': match_info['season_name'],
                    'home_team': match_info['home_team'],
                    'away_team': match_info['away_team'],

                    'timestamp': event['timestamp'],
                    'goal_minute': goal['minute'],

                    'event_id': event['id'],
                    'event_type': event['type'],
                    'event_minute': event['minute'],
                    'event_second': event['second'],

                    'team': event['team'],
                    'player': event.get('player', None),
                    'location': event.get('location', None),

                    'possession': event['possession'],
                    'possession_team': event['possession_team'],

                    'play_pattern': event.get('play_pattern', None),

                    'pass_type': event.get('pass_type', None),
                    'pass_outcome': event.get('pass_outcome', None),
                    'pass_length': event.get('pass_length', None),
                    'pass_recipient': event.get('pass_recipient', None),
                    'pass_end_location': event.get('pass_end_location', None),
                    'pass_angle': event.get('pass_angle', None),
                    'pass_height': event.get('pass_height', None),
                    'pass_cross': event.get('pass_cross', None),
                    'pass_straight': event.get('pass_straight', None),
                    'pass_switch': event.get('pass_switch', None),
                    'pass_through_ball': event.get('pass_through_ball', None),
                    'pass_type': event.get('pass_type', None),



                    'dribble_outcome': event.get('dribble_outcome', None),
                    'shot_outcome': event.get('shot_outcome', None),
                    'shot_statsbomb_xg': event.get('shot_statsbomb_xg', None),
                    'shot_end_location': event.get('shot_end_location', None),

                    'duel_outcome': event.get('duel_outcome', None),
                    'duration': event.get('duration', None),
                    'foul_committed_card': event.get('foul_committed_card', None),

                    'goalkeeper_outcome': event.get('goalkeeper_outcome', None),

                    'under_pressure': event.get('under_pressure', None),

                    'carry_end_location': event.get('carry_end_location', None),

                    'half': half,
                    'team_started_half': team_started_half,
                    'team_scored': team_scored,
                }
                goal_event_data.append(event_info)

            teams_involved = [match_info['home_team'], match_info['away_team']]
            for team in teams_involved:
                surrounding = get_surrounding_matches(team, match_info['match_date'])
                previous_match_ids = surrounding['previous_matches']
                next_match_ids = surrounding['next_matches']
                related_match_ids = previous_match_ids + next_match_ids

                for related_match_id in related_match_ids:
                    try:
                        related_events = sb.events(match_id=related_match_id, flatten_attrs=True)
                        related_events['minute'] = pd.to_numeric(related_events['minute'], errors='coerce').fillna(0).astype(int)
                        related_events['second'] = pd.to_numeric(related_events['second'], errors='coerce').fillna(0).astype(int)
                    except Exception as e:
                        print(f"Error retrieving events for related match_id {related_match_id}: {e}")
                        continue

                    related_match_info = EG_all_matches[EG_all_matches['match_id'] == related_match_id]

                    if related_match_info.empty:
                        print(f"No match info found for related_match_id {related_match_id}. Skipping.")
                        continue

                    related_match_info = related_match_info.iloc[0]
                    related_home = related_match_info['home_team']
                    related_away = related_match_info['away_team']

                    metrics = calculate_team_metrics(related_events, related_home, related_away)

                    for team_name, team_metrics in metrics.items():
                        metrics_record = {
                            'original_match_id': match_id,
                            'original_goal_minute': goal['minute'],
                            'original_competition_name': match_info['competition_name'],
                            'original_season_name': match_info['season_name'],
                            'original_half': half,
                            'original_home_team': match_info['home_team'],
                            'original_away_team': match_info['away_team'],
                            'original_scoring_team': team_scored,
                            'related_match_id': related_match_id,
                            'related_team': team_name,
                            'GF': team_metrics['GF'],
                            'GA': team_metrics['GA'],
                            'Shots_on_Target': team_metrics['Shots_on_Target'],
                            'Shots_off_Target': team_metrics['Shots_off_Target'],
                            'Possession_Percentage': team_metrics['Possession_Percentage'],
                            'Pass_Accuracy_Percentage': team_metrics['Pass_Accuracy_Percentage'],
                            'xG': team_metrics['xG'],
                            'xGA': team_metrics['xGA']
                        }
                        team_metrics_data.append(metrics_record)

df_goals = pd.DataFrame(goal_data)
df_goal_events = pd.DataFrame(goal_event_data)
df_team_metrics = pd.DataFrame(team_metrics_data)

df_goals.to_csv('goals.csv', index=False)
df_goal_events.to_csv('goal_events.csv', index=False)
df_team_metrics.to_csv('team_metrics_surrounding.csv', index=False)

print("Goals DataFrame:")
print(df_goals.head())

print("\nGoal Events DataFrame:")
print(df_goal_events.head())

print("\nTeam Metrics Surrounding DataFrame:")
print(df_team_metrics.head())


Processing match_id: 3890511
team: Hertha Berlin
team: Hannover 96
Processing match_id: 3890384
team: Borussia Dortmund
team: VfB Stuttgart
Processing match_id: 3890282
team: FSV Mainz 05
team: Hannover 96
Processing match_id: 3890540
team: Bayer Leverkusen
team: Hertha Berlin
Processing match_id: 3890535
team: Wolfsburg
team: Augsburg
Processing match_id: 3890533
team: Hertha Berlin
team: Bayern Munich
Processing match_id: 3890531
team: FC Köln
team: Darmstadt 98
Processing match_id: 3890527
team: FSV Mainz 05
team: FC Köln
Processing match_id: 3890520
team: Hannover 96
team: Borussia Mönchengladbach
Processing match_id: 3890518
team: Schalke 04
team: Borussia Dortmund
Processing match_id: 3890500
team: VfB Stuttgart
team: Bayer Leverkusen
Processing match_id: 3890486
team: Hoffenheim
team: Wolfsburg
Processing match_id: 3890485
team: Ingolstadt
team: VfB Stuttgart
Processing match_id: 3890480
team: FC Köln
team: Schalke 04
Processing match_id: 3890473
team: Schalke 04
team: Hamburger

In [ ]:
df_goals.to_csv('/content/drive/MyDrive/Hackathon FUT/Data/EG_goals.csv', index=False)
df_goal_events.to_csv('/content/drive/MyDrive/Hackathon FUT/Data/EG_events.csv', index=False)
df_team_metrics.to_csv('/content/drive/MyDrive/Hackathon FUT/Data/EG_team_metrics_surrounding.csv', index=False)